In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
page1 = requests.get("https://www.imdb.com/search/title/?groups=top_250&sort=user_rating")
page2 = requests.get("https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=51&ref_=adv_nxt")
page3 = requests.get("https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=101&ref_=adv_nxt")
page4 = requests.get("https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=151&ref_=adv_nxt")
page5 = requests.get("https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=201&ref_=adv_nxt")
pages = [page1, page2, page3, page4, page5]

In [3]:
headers = ['Titre', 'Année', 'Score', 'Réalisateur', 'Votes', 'Recette']
movies = pd.DataFrame(columns=headers)

for page in pages:
    soup = BeautifulSoup(page.content, 'html.parser')
    films = soup.find_all("div", {"class":"lister-item-content"})
    for film in films:
        row = []
        title = film.h3.a.text
        row.append(title)
        year = film.h3.find("span", {"class":"lister-item-year text-muted unbold"}).text.strip('I ()')
        row.append(year)
        rate = film.find("div", {"class":"ratings-imdb-rating"}).strong.text
        row.append(rate)
        director = film.find_all('p')[2].a.text
        row.append(director)
        for i in range(5):
            if i % 5 == 1:
                vote = film.find("p", {"class":"sort-num_votes-visible"}).find_all("span")[1]["data-value"]
                row.append(vote)
            elif i % 5 == 4:
                if len(film.find("p", {"class":"sort-num_votes-visible"}).find_all("span")) < 4:
                    row.append("0")
                else:
                    gross = film.find("p", {"class":"sort-num_votes-visible"}).find_all("span")[4]["data-value"]
                    row.append(gross)
            else:
                pass
        length = len(movies)
        movies.loc[length] = row

In [4]:
movies

,Titre,Année,Score,Réalisateur,Votes,Recette
0,Les Évadés,1994,9.3,Frank Darabont,2384390,"28,341,469"
1,Le parrain,1972,9.2,Francis Ford Coppola,1652481,"134,966,411"
2,The Dark Knight : Le Chevalier noir,2008,9.0,Christopher Nolan,2347860,"534,858,444"
3,"Le parrain, 2ème partie",1974,9.0,Francis Ford Coppola,1149835,"57,300,000"
4,12 Hommes en colère,1957,9.0,Sidney Lumet,704006,"4,360,000"
...,...,...,...,...,...,...
245,Mr. Smith au sénat,1939,8.1,Frank Capra,108449,"9,600,000"
246,Autant en emporte le vent,1939,8.1,Victor Fleming,293969,"198,676,459"
247,New York - Miami,1934,8.1,Frank Capra,95743,"4,360,000"
248,La passion de Jeanne d'Arc,1928,8.1,Carl Theodor Dreyer,48979,"21,877"
